In [3]:
from binance.client import Client
import pandas as pd
import gym
from stable_baselines3 import A2C,PPO,DQN
from stable_baselines3.ppo.policies import MlpPolicy
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.callbacks import CallbackList
from stable_baselines3.common.vec_env import DummyVecEnv
import matplotlib.pyplot as plt
import numpy as np
import os
from stable_baselines3.common.logger import configure
import pandas_ta as ta
from pon import api,secret
import warnings
warnings.filterwarnings("ignore")

AttributeError: partially initialized module 'gym' has no attribute 'core' (most likely due to a circular import)

In [95]:
def get_price_data_binance(ticker:str,limit:int,start_str:str, end_str:str,interval = Client.KLINE_INTERVAL_1DAY)->pd.DataFrame:
    client = Client(api, secret)
    df = pd.DataFrame(client.get_historical_klines(symbol=ticker, interval=interval,start_str=start_str, end_str=end_str))
    df.columns=['date','Open','High','Low','Close','Volume','close_time','d1','d2','d3','d4','d5']
    df = df.drop(['close_time','d1','d2','d3','d4','d5'],axis=1)
    df['date'] = pd.to_datetime(df['date']*1000000)
    df['Open'] = df['Open'].astype(float)
    df['High'] = df['High'].astype(float)
    df['Low'] = df['Low'].astype(float)
    df['Close'] = df['Close'].astype(float)
    df.set_index('date',inplace=True)
    df['Volume'] = df['Volume'].apply(lambda x:round(float(x),2))
    return df


In [96]:
traindf = get_price_data_binance('BTCUSDT',1000,'1 Jan, 2022','30 Nov, 2022',interval = Client.KLINE_INTERVAL_1HOUR)
testdf = get_price_data_binance('BTCUSDT',1000,'1 Dec, 2022','31 Dec, 2022',interval = Client.KLINE_INTERVAL_1HOUR)

In [112]:
env_maker = lambda: gym.make('stocks-v0',df=traindf,frame_bound=(200,7900),window_size=200)
env = DummyVecEnv([env_maker])
#model = DQN.load(os.path.join('models','DQN_YEAR.zip'),env=env)
model = DQN('MlpPolicy',env,verbose=1)
tmp_path = os.path.join('logs','DQN_YEAR')
new_logger = configure(tmp_path, ["tensorboard"])
model.set_logger(new_logger)
model.learn(total_timesteps=10000000)
model.save(os.path.join('models','DQN_YEAR'))

TypeError: DQN.learn() got an unexpected keyword argument 'timesteps'

In [111]:
#model = PPO.load(os.path.join('models','PPO_1M_RECENT.zip'))
# del model
model.num_timesteps

100000

In [1]:
env = gym.make('stocks-v0',df=testdf,frame_bound=(200,700),window_size=200)

obs = env.reset()
while True:
    obs=obs[np.newaxis, ...]
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    if done:
        print('info',info)
        break
plt.figure(figsize=(15,6))
plt.cla()
env.render_all()
plt.show()

NameError: name 'gym' is not defined